## Party Strength

There seems to be no panel dataset of party strength in each state so here I attempt to scrape results from Wikipedia. It's a start and likely also the end. 

In [1]:
import os, re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import us

data_path = "C:/Users/SpiffyApple/Documents/USC/RaphaelBostic/policy_diffusion"

In [2]:
#defining some preliminary functions: 
def fetch_website(url):
    """
    To hide that the scraping is being done via Python, I change the user-agent to a Firefox
    browser so that the website believes it is a chrome browser accessing them. Hope it works.
    """
    user_agent={'User-agent':'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.18 Safari/537.36'}
    r=requests.get(url, headers=user_agent)
    try:
        #print("Accessed and downloaded URL data")
        return(r.content)
    except ConnectionError:
        print("Skipping this url")
        return(None)

In [3]:
##get US states to loop over
us_states = [state.name for state in us.STATES if state.name != "District of Columbia"]
us_states.append("Washington, D.C.") #fix a little wikipedia issue

In [4]:
##fwtch wiki data and record ones that don't have it
out_dict = {}
no_wiki = []
missing_text = "Wikipedia does not have an article with this exact name."
url_base = "https://en.wikipedia.org/wiki/Political_party_strength_in_"
wiki_urls = [url_base + re.sub(" ", "_",state) for state in us_states]

for state in us_states:
    out_dict[state] = bs(fetch_website(url_base + re.sub(" ", "_",state)), 'lxml')
    if missing_text in out_dict[state].getText():
        no_wiki.append(state)
        out_dict[state] = None

In [5]:
no_wiki 

[]

### Find the desired tables
This is simply done by looking at which tables on the corresponding Wikipedia page contain the year 2014 since that's the year most other datasets do have. 

In [6]:
#usually there are multiple tables. Pick out ones that contain 2014
data_dict = {}
for state in us_states:
    found_tables = out_dict[state].findAll("table", class_='wikitable')
    for table in found_tables:
        if "2014" in table.getText():
            data_dict[state] = table
            
#check how many states had a table with '2014' in it
print("Number of states with wikipedia table for party strength: %d" %len(data_dict.keys()))         

Number of states with wikipedia table for party strength: 51


### Parse the tables into Pandas DataFrames

In [7]:
## corresponding functiosn:
def pre_process_table(table):
    """
    INPUT:
        1. table - a bs4 element that contains the desired table: ie <table> ... </table>
    OUTPUT:
        a tuple of: 
            1. rows - a list of table rows ie: list of <tr>...</tr> elements
            2. num_rows - number of rows in the table
            3. num_cols - number of columns in the table
    Options:
        include_td_head_count - whether to use only th or th and td to count number of columns (default: False)
    """
    rows = [x for x in table.find_all('tr')]

    num_rows = len(rows)
    
    num_cols = max([len(x.find_all(['th','td'])) for x in rows])

    ##lets make col_num counter more complicated to account for colspans in headers. 
    header_rows_set = [x.find_all(['th', 'td']) for x in rows if len(x.find_all(['th', 'td']))>num_cols/2]
    
    num_cols_set = []

    for header_rows in header_rows_set:
        num_cols = 0
        for cell in header_rows:
            row_span, col_span = get_spans(cell)
            num_cols+=len([cell.getText()]*col_span)
            
        num_cols_set.append(num_cols)
    
    num_cols = max(num_cols_set)
    #print(num_cols)
    
    return (rows, num_rows, num_cols)


def get_spans(cell):
        """
        INPUT:
            1. cell - a <td>...</td> or <th>...</th> element that contains a table cell entry
        OUTPUT:
            1. a tuple with the cell's row and col spans
        """
        if cell.has_attr('rowspan'):
            rep_row = int(cell.attrs['rowspan'])
        else: # ~cell.has_attr('rowspan'):
            rep_row = 1
        if cell.has_attr('colspan'):
            rep_col = int(cell.attrs['colspan'])
        else: # ~cell.has_attr('colspan'):
            rep_col = 1 
        
        return (rep_row, rep_col)
 
def process_rows(rows, num_rows, num_cols):
    """
    INPUT:
        1. rows - a list of table rows ie <tr>...</tr> elements
    OUTPUT:
        1. data - a Pandas dataframe with the html data in it
    """
    data = pd.DataFrame(np.ones((num_rows, num_cols))*np.nan)
    for i, row in enumerate(rows):
        try:
            col_stat = data.iloc[i,:][data.iloc[i,:].isnull()].index[0]
        except IndexError:
            print("Error at row %d" %i, row, "\nParser may have failed to acquire correct number of columns. Num of columns: %d" %num_cols)
            print("Failed to locate starting point in above row. Subsequent rows are likely to be erronously parsed")
            
        for j, cell in enumerate(row.find_all(['td', 'th'])):
            rep_row, rep_col = get_spans(cell)

            #print("cols {0} to {1} with rep_col={2}".format(col_stat, col_stat+rep_col, rep_col))
            #print("\trows {0} to {1} with rep_row={2}".format(i, i+rep_row, rep_row))

            #find first non-na col and fill that one
            while any(data.iloc[i,col_stat:col_stat+rep_col].notnull()):
                col_stat+=1

            data.iloc[i:i+rep_row,col_stat:col_stat+rep_col] = cell.getText()
            if col_stat<data.shape[1]-1:
                col_stat+=rep_col

    return data

def main(table):
    rows, num_rows, num_cols = pre_process_table(table)
    df = process_rows(rows, num_rows, num_cols)
    
    return(df)

In [36]:
## loop through the ditionary:
df_dict_origin = {}

for state in us_states:
    df_dict_origin[state] = main(data_dict[state])

### Clean and combine the 51 data tables
First, perhaps make each datatable into an indexed frame and then work from there. 

In [37]:
df_dict = df_dict_origin.copy()

##it may actually make sense to make columns from the second to last row, then drop duplicates if necessary, set index, etc
for state in us_states:
    df_dict[state].columns = df_dict[state].iloc[-1].str.lower() + ":" + df_dict[state].iloc[-2].str.lower()
    df_dict[state].columns.name = None
    
    #remove duplicated columns:
    df_dict[state] = df_dict[state].loc[:,~df_dict[state].columns.duplicated()]
    
    #set the index, clean it up, assign it the right numeric type, subset
    df_dict[state].set_index('year:year', inplace=True)
    df_dict[state] = df_dict[state][df_dict[state].index.str.contains("\d")]
    df_dict[state].index.name = 'year'
    
    df_dict[state].index = df_dict[state].index.str.extract("(\d+)", expand=False)
    df_dict[state].index = df_dict[state].index.astype(np.int64)
    df_dict[state] = df_dict[state].loc[df_dict[state].index>=1980]
    
    df_dict[state] = df_dict[state].dropna(axis=1, how='all')

In [42]:
# try to fix the columns -- this cell may need to be run multiple times -- not sure why
for state in us_states:
    df_dict[state].columns = df_dict[state].columns.str.replace("^gov$", "governor")
    df_dict[state].columns = df_dict[state].columns.str.replace("general house assembly", 'state legislature')
    df_dict[state].columns = df_dict[state].columns.str.replace("general court", 'state legislature')
    df_dict[state].columns = df_dict[state].columns.str.replace("general assembly", "state legislature")    
    df_dict[state].columns = df_dict[state].columns.str.replace("/constitutional", "")
    df_dict[state].columns = df_dict[state].columns.str.replace("mayor", 'governor')   
    df_dict[state].columns = df_dict[state].columns.str.replace(" assem$", ' house')
    df_dict[state].columns = df_dict[state].columns.str.replace(" assembly$", ' house')
    df_dict[state].columns = df_dict[state].columns.str.replace("district \d$", "")    
    df_dict[state].columns = df_dict[state].columns.str.replace("representatives", "house")
    df_dict[state].columns = df_dict[state].columns.str.replace("legislative assembly", "state legislature")
    
    df_dict[state].columns = df_dict[state].columns.str.replace("united states", 'us')
    
    df_dict[state].columns = df_dict[state].columns.str.replace("class i$", "class 1")
    df_dict[state].columns = df_dict[state].columns.str.replace("class ii$", "class 2")
    df_dict[state].columns = df_dict[state].columns.str.replace("class iii$", "class 3")
    
    df_dict[state].columns = df_dict[state].columns.str.replace("senate", "sen")
    df_dict[state].columns = df_dict[state].columns.str.replace("senator", 'sen')
    
    df_dict[state].columns = df_dict[state].columns.str.replace("^us |:us| u\.s\.", " ")
    df_dict[state].columns = df_dict[state].columns.str.replace(":state", " ")
    #df_dict[state].columns = df_dict[state].columns.str.replace("lieutenant", 'lt')
    #df_dict[state].columns = df_dict[state].columns.str.replace("(?:s)", "\1")
    df_dict[state].columns = df_dict[state].columns.str.replace("  ", " ")
    df_dict[state].columns = df_dict[state].columns.str.replace("executive offices?:", "")
    #df_dict[state].columns = df_dict[state].columns.str.replace("^electoral college votes:", "")

    df_dict[state].columns = df_dict[state].columns.str.replace("\(|\)|:|\\n", " ")
    df_dict[state].columns = df_dict[state].columns.str.replace("\.", "")
    df_dict[state].columns = df_dict[state].columns.str.replace("\scollege([\s\w]+)", "")
    df_dict[state].columns = df_dict[state].columns.str.replace("\sus\s", " ")
    df_dict[state].columns = df_dict[state].columns.str.replace("state legislature legislature", "state legislature house")
    df_dict[state].columns = df_dict[state].columns.str.strip()

In [43]:
df_dict['Washington, D.C.'].rename(columns = {'congress house delegate':"congress house",
                                              "congress shadow sen seat 1":"congress sen class 1",
                                             'congress shadow sen seat 2':"congress sen class 2",
                                             'presidential electoral':"electoral"}, inplace=True)
df_dict['Washington, D.C.'].drop("congress shadow representative", axis=1, inplace=True)
df_dict['Maine'].drop("congress former house districts", axis=1, inplace=True)

In [44]:
##some columns are now duplicated, need to rid myself of them. Similar with the senate columns.
for state in us_states:
    bool_arr = df_dict[state].columns.str.contains("congress house")
    if np.sum(bool_arr)>1:
        temp = df_dict[state].loc[:,df_dict[state].columns.str.contains("congress house")].sum(axis=1)
        df_dict[state] = df_dict[state].loc[:,~bool_arr]
        df_dict[state]['congress house'] = temp

In [45]:
for state in us_states:
    print(state+"\n",df_dict[state].loc[:,df_dict[state].columns.str.contains('^governor|house|sen|electoral')].shape[1])

Alabama
 7
Alaska
 7
Arizona
 7
Arkansas
 7
California
 7
Colorado
 7
Connecticut
 7
Delaware
 7
Florida
 7
Georgia
 7
Hawaii
 7
Idaho
 7
Illinois
 7
Indiana
 7
Iowa
 7
Kansas
 7
Kentucky
 7
Louisiana
 7
Maine
 7
Maryland
 7
Massachusetts
 6
Michigan
 7
Minnesota
 7
Mississippi
 7
Missouri
 7
Montana
 7
Nebraska
 6
Nevada
 7
New Hampshire
 7
New Jersey
 7
New Mexico
 7
New York
 7
North Carolina
 7
North Dakota
 7
Ohio
 7
Oklahoma
 7
Oregon
 7
Pennsylvania
 7
Rhode Island
 7
South Carolina
 7
South Dakota
 7
Tennessee
 7
Texas
 7
Utah
 7
Vermont
 7
Virginia
 7
Washington
 7
West Virginia
 7
Wisconsin
 7
Wyoming
 7
Washington, D.C.
 5


In [46]:

combnd = pd.concat([df_dict[state].loc[:,df_dict[state].columns.str.contains('^governor|house|sen|electoral')] for state in us_states], keys=us_states)

In [47]:
combnd.head()

congress house congress sen class 1 congress sen class 2  \
        year                                                            
Alabama 1980         4D, 3R                  NaN    Howell Heflin (D)   
        1981         4D, 3R                  NaN    Howell Heflin (D)   
        1982         4D, 3R                  NaN    Howell Heflin (D)   
        1983         5D, 2R                  NaN    Howell Heflin (D)   
        1984         5D, 2R                  NaN    Howell Heflin (D)   

               congress sen class 3  \
        year                          
Alabama 1980  Donald W. Stewart (D)   
        1981    Jeremiah Denton (R)   
        1982    Jeremiah Denton (R)   
        1983    Jeremiah Denton (R)   
        1984    Jeremiah Denton (R)   

                                              electoral            governor  \
        year                                                                  
Alabama 1980  Ronald Reagan and George H. W. Bush (R) Y       Fob James (D)   
        1981  Ronald Reagan and George H. W. Bush (R) Y       Fob James (D)   
        1982  Ronald Reagan and George H. W. Bush (R) Y       Fob James (D)   
        1983  Ronald Reagan and George H. W. Bush (R) Y  George Wallace (D)   
        1984  Ronald Reagan and George H. W. Bush (R) Y  George Wallace (D)   

             state legislature house state legislature sen  
        year                                                
Alabama 1980                101D, 4R                   35D  
        1981                101D, 4R                   35D  
        1982                101D, 4R                   35D  
        1983                 97D, 8R               32D, 3R  
        1984                87D, 18R           29D, 3R, 3I

In [48]:
combnd.to_csv("/".join([data_path, "states_party_strength.csv"]))